In [2]:
import spacy

In [3]:
#!python -m spacy download en_core_web_md

     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     --------------------------------------- 0.0/42.8 MB 330.3 kB/s eta 0:02:10
     --------------------------------------- 0.0/42.8 MB 325.1 kB/s eta 0:02:12
     --------------------------------------- 0.0/42.8 MB 325.1 kB/s eta 0:02:12
     --------------------------------------- 0.0/42.8 MB 217.9 kB/s eta 0:03:17
     --------------------------------------- 0.1/42.8 MB 374.1 kB/s eta 0:01:55
     --------------------------------------- 0.1/42.8 MB 437.6 kB/s eta 0:01:38
     --------------------------------------- 0.1/42.8 MB 379.3 kB/s eta 0:01:53
     --------------------------------------- 0.2/42.8 MB 513.4 kB/s eta 0:01:23
     --------------------------------------- 0.2/42.8 MB 628.1 kB/s eta 0:01:08
     --------------------------------------- 0.3/42.8 MB 561.1 kB/s eta 0:01:16
     --------------------------------------- 0.3/42.8 MB 613.6 kB/s eta 0:01:10
     --------------------------------------- 0.4

In [4]:
import pandas as pd
pd.options.display.max_colwidth = None
pd.options.display.max_rows = 6
data = pd.read_csv("spacy_textcat/reddit_data.csv")
data

,text,tag,subreddit,id
0,"I’m looking for datasets or api source that quantifies fan base, or preferably, bettors’ sentiment regarding a team’s performance or direction. Does anyone know of an API that tracks this? For now I’m looking specifically for NBA, but am also interested in MLB, NFL, and NCAA f-ball and b-ball.",API,datasets,s0vufk
1,"I'm making an ESG stock analysis program in Java, and so far the only free ESG API I've come across is ESGEnterprise, but I'm having trouble retrieving the data. Has anyone had any success/have any recs for other ESG APIs out there.",API,datasets,ruvj9n
2,"Hey everyone! I’m one of the creators of Sieve _URL_ and I’m excited to be sharing it!\nSieve _URL_ **is an API that helps you turn petabyte-scale video data into a high-quality dataset, automatically.**\nIt helps store, process, and semantically search your video data. Just think _NUMBER_ cameras recording footage at _NUMBER_ FPS, _NUMBER_/_NUMBER_. That would be _NUMBER_ million frames generated in a single day. The videos might be searchable by timestamp, but finding moments of interest is like searching for a needle in a haystack. Sieve tags useful attributes like people, motion, lighting, etc on every frame!\nWe built this visual demo (link here _URL_ a little while back which we’d love to get feedback on. It’s \~_NUMBER_ hours of security footage that our API processed in <_NUMBER_ mins and has simple querying and export functionality enabled. We see applications in better understanding what data you have, figuring out which data to send to labeling, sampling datasets for training, and building multiple test sets for models by scenario.\nTo try it on your videos: _URL_ _URL_\nVisual dashboard walkthrough: Click on our site link!",API,datasets,rup1uj
...,...,...,...,...
718,"I'm currently in the process of learning NLP. I am using catalyst on c#. \nI was able to run the sample programs and it was able to determine if the word is an noun, adjective, etc. But I can't find any sample for what I need.\nHere is a summary of what I would like to achieve. \nI would like to extract certain information on a sentence. Lets say i have the following texts:\n""Sally ate an orange this morning. ""\nOr \n""Sally is hiding behind the cabinet and she is eating an orange. "" \nHow do i use the nlp to extract what sally ate?",NaN,LanguageTechnology,saas64
719,"I’ve been trying to do some basic keyword extraction and finding it harder than expected.\nKeyBERT seems good but it requires a powerful GPU to be usably fast. That’s possible with AWS, but there’s a bit more set up.\nI just tried PyTextRank, and I was surprised at the quality of the output - I wouldn’t say it was perfect either. Maybe I should set a threshold, like choose the top _NUMBER_ ranked keywords? It’s fine if we exclude potential good keywords just to have a smaller list of good ones.\nHere’s a good article about _NUMBER_ different methods, which is helpful -\n_URL_\nIn theory, Spacy and BERT seem like the best options but they’re both a little complex. \nI think KW extraction really only needs a few layers or as Spacy would call them pipelines.\n_NUMBER_. accurate tokenization of words and punctuation symbols\n_NUMBER_. accurate recognition of multi-word expressions - think of it as “chunking”\n_NUMBER_. Strong assessment of keyword “candidacy” for each MWE \nOf course, a good algorithm can often skip steps. Like BERT is so smart it doesn’t need anything but the input text.\nDoes anyone know of a simplest way to run a fast, effective keyword extraction?\nI’m talking _NUMBER_ keywords in one second on a fast CPU.\nThanks very much",NaN,LanguageTechnology,s7qml8
720,This _URL_ position is currently open and I wanted to share with you!,NaN,LanguageTechnology,s30ccv


In [7]:
#for the text classification component
cats = data.subreddit.unique().tolist()
cats

['datasets', 'dataengineering', 'LanguageTechnology']

In [8]:
from typing import Set, List, Tuple
from spacy.tokens import DocBin
import spacy

nlp = spacy.load("en_core_web_md")

def make_docs(data: List[Tuple[str, str]], target_file: str, cats: Set[str]):
    docs = DocBin()
    # Use nlp.pipe to efficiently process a large number of text inputs,
    # the as_tuple arguments enables giving a list of tuples as input and
    # reuse it in the loop, here for the labels
    for doc, label in nlp.pipe(data, as_tuples=True):
        # Encode the labels (assign 1 the subreddit)
        for cat in cats:
            doc.cats[cat] = 1 if cat == label else 0
        docs.add(doc)
    docs.to_disk(target_file)
    return docs

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(data["text"].values, data["subreddit"].values, test_size=0.3)

# make_docs(list(zip(X_train, y_train)), "train.spacy", cats=cats)
# make_docs(list(zip(X_valid, y_valid)), "valid.spacy", cats=cats)

504 504


In [11]:
from spacy.cli.train import train as spacy_train
config_path = "spacy_textcat/config.cfg"
output_model_path = "output/spacy_textcat"
spacy_train(
    config_path,
    output_path=output_model_path,
    overrides={
        "paths.train": "train.spacy",
        "paths.dev": "valid.spacy",
    },
)

✔ Created output directory: output\spacy_textcat
ℹ Saving to output directory: output\spacy_textcat
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.22        3.59    0.04
  0     200         37.56       37.31    0.37
  0     400         26.53       55.96    0.56
  1     600          7.89       74.00    0.74
  1     800          9.21       79.53    0.80
  2    1000          2.57       83.73    0.84
  3    1200          1.48       75.15    0.75
  4    1400          1.82       80.75    0.81
  5    1600          0.63       81.07    0.81
  6    1800          0.91       79.96    0.80
  7    2000          0.53       79.91    0.80
  9    2200          0.52       81.68    0.82
 11    2400